In [ ]:
from steam import Steam
from decouple import config

with open("myapikey.txt", "r") as file:
    myapikey=file.read()

import os
os.environ["STEAM_API_KEY"] = myapikey
KEY = config("STEAM_API_KEY")
steam = Steam(KEY)

In [ ]:
# This cell generates the random User ID's
import random
from time import sleep
import pickle

num_ids=500  # number of random user ID's to sample

id_set = set()
while len(id_set) < num_ids:
    temp = str(76561198000000000 + random.randint(-10000000, 100000000))
    
    # Check that user ID has not already been used before writing to a set
    if temp not in id_set:
        id_set.add(temp)
        
id_list = list(id_set)

In [ ]:
# This cell iterates over the random user ID's, queries the api, and periodically
# writes output to a file. 

valid_data = []
valid_ids = []
file_num = 0  # tracks number of files outputted so far.
users_in_file = 10  # number of user's data for each file

for j in range(len(id_list)):
    # Pull owned games from the api
    temp = steam.users.get_owned_games(id_list[j])
    
    # assuming output is nonempty, write to the "valid_data" list
    if len(temp) != 0:
        valid_data.append([id_list[j], temp])
        valid_ids.append(id_list[j])
    if (j % int(num_ids/100)) == 0:
        print('progress: {:.2f}%'.format(100*j/num_ids))
    sleep(1)
    
    # Periodically write to a file, in case of crashes, etc.
    if len(valid_data) == users_in_file:
        with open('./raw_data/game_data_{}.pkl'.format(file_num), 'wb') as fp:
            pickle.dump(valid_data, fp)
        valid_data = []
        file_num+=1

# Write leftover data to a last file
with open('./raw_data/game_data_{}.pkl'.format(file_num), 'wb') as fp:
    pickle.dump(valid_data, fp)
    
with open('valid_ids.pkl', 'wb') as fp:
    pickle.dump(valid_ids, fp)    

In [ ]:
# This is the syntax to load the data from the pkl files. 
# loaded_data will be the name of the python list with the data from the file.
with open ('./raw_data/game_data_0.pkl', 'rb') as fp:
    loaded_data = pickle.load(fp)
loaded_data